In [1]:
%%capture
#from jupyterthemes import jtplot
#jtplot.style()
import numpy as np
from landlab import FieldError
from landlab.utils import get_watershed_mask
import xarray as xr
import pandas as pd
import scipy
import gdal
from scipy import ndimage
from scipy.ndimage.filters import *
import os
import math
from osgeo import osr
from fractions import Fraction
import timeit
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
%matplotlib inline
import random
# import plotting tools
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib as mpl
from landlab.plot.imshow import imshow_grid 

# import necessary landlab components
from landlab import RasterModelGrid, HexModelGrid
from landlab.components import FlowAccumulator, LakeMapperBarnes, SinkFillerBarnes
from landlab.components import(FlowDirectorD8, 
                               FlowDirectorDINF, 
                               FlowDirectorMFD, 
                               FlowDirectorSteepest)
from landlab.components import DepressionFinderAndRouter
# import landlab plotting functionality
from landlab.plot.drainage_plot import drainage_plot
from pylab import show, figure



In [2]:
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

input_DEM_resolution = 1
working_resolution = 1
location = (r'C:\PhD\junk')
os.chdir(location)
input_geotiff = gdal.Open('aranda_subset.tif')
x = np.array(input_geotiff.GetRasterBand(1).ReadAsArray())
#input_DEM = scipy.ndimage.zoom(x, input_DEM_resolution / working_resolution, order = 1)
empty_grid_rows = roundup(x.shape[0])
empty_grid_cols = roundup(x.shape[1])
input_DEM = np.zeros([empty_grid_rows,empty_grid_cols])
input_DEM[:x.shape[0], :x.shape[1]] = x

In [3]:
input_DEM.shape

(1480, 1070)

In [4]:
def rectangleWindow(m, n):
    """Takes a value for number of rows (m) and number of columns (n) such that
       m and n are both positive real numbers and creates a rectangle of 
       boolian 'True' values."""
    rectangle = np.ones((m, n), dtype=bool) 
    return rectangle
def number_of_values(Window):
    """This funciton takes the shape function as an input and returns a number 
        of values present in the specified shape. 
        
        This can be different for a different window shape and to initialize
        requires the specification of the function for the given window type and 
        parameter values required for that input function.
        
        To initialize this function for shape == rectangle type 'number_of_values(rectangleWindow(m,n)) 
        where m and n are any positive real number as per the rectangleWindow function."""
    denominator = sum(sum(Window > 0))
    return denominator
def slopeWindow(DEM_slope, x_cellsize, y_cellsize):
    """This function implements slope calculation using the same algorithm
       as ARCGIS (Outlined on the page 'How Slope Works').
       This particular example of the function is written such that it
       will only work if called within the ndimage generic_filter (as the first input).
       This is because the index arguments for a-e are given for the 1d array created
       by the generic_filter function after extracting values from the 3,3 rectangle window.
       NOTE: THIS FUNCTION ONLY WORKS WITH A 3x3 RECTANGLE WINDOW."""
    a = DEM_slope[0]; b = DEM_slope[1]; c = DEM_slope[2]
    d = DEM_slope[3]; e = DEM_slope[4]; f = DEM_slope[5]
    g = DEM_slope[6]; h = DEM_slope[7]; i = DEM_slope[8]
    dzdx = ((c + (2*f) + i) - (a + (2*d) + g)) / (8 * x_cellsize)
    dzdy = ((g + (2*h) + i) - (a + (2*b) + c)) / (8 * y_cellsize)
    rise_run = np.sqrt(dzdx**2 + dzdy**2)
    slope_degrees = np.arctan(rise_run) * (180/math.pi)
    slope_percent = rise_run * 100
    #Can also ask it to return slope_degrees but askinh for both causes it to throw and error.
    return slope_percent
# The 'extra_arguments' variable requires a value that represents r in the PCTL function defined above.
# The reason it need to be assigned to a seperate variable is that the generic_filter function only allows the
# input function (PCTL in this case) to take one argument (S1). Then, if the input function normally 
# takes more than one argument the 'extra_arguments' variable needs to be defined as a tuple 
# (hence (3,) instead of (3)).
def slopeWindowDegrees(DEM, x_cellsize, y_cellsize):
    """This function implements slope calculation using the same algorithm
       as ARCGIS (Outlined on the page 'How Slope Works').
       This particular example of the function is written such that it
       will only work if called within the ndimage generic_filter (as the first input).
       This is because the index arguments for a-e are given for the 1d array created
       by the generic_filter function after extracting values from the 3,3 rectangle window.
       NOTE: THIS FUNCTION ONLY WORKS WITH A 3x3 RECTANGLE WINDOW."""
    a = DEM[0]; b = DEM[1]; c = DEM[2]
    d = DEM[3]; e = DEM[4]; f = DEM[5]
    g = DEM[6]; h = DEM[7]; i = DEM[8]
    dzdx = ((c + (2*f) + i) - (a + (2*d) + g)) / (8 * x_cellsize)
    dzdy = ((g + (2*h) + i) - (a + (2*b) + c)) / (8 * y_cellsize)
    rise_run = np.sqrt(dzdx**2 + dzdy**2)
    slope_degrees = np.arctan(rise_run) * (180/math.pi)
    slope_percent = rise_run * 100
    #Can also ask it to return slope_degrees but asking for both causes it to throw and error.
    return slope_degrees
def planCurvature(DEM, cellsize):
    """This process is taken from Change (2014, Introduction to Geographic Information
    systems, Page 284)."""
    Z1 = DEM[0]; Z2 = DEM[1]; Z3 = DEM[2]
    Z4 = DEM[3]; Z0 = DEM[4]; Z5 = DEM[5]
    Z6 = DEM[6]; Z7 = DEM[7]; Z8 = DEM[8]
    D = (((Z4 + Z5)/2) - Z0) / cellsize**2
    E = (((Z2 + Z7)/2) - Z0) / cellsize**2
    F = (Z3 - Z1 + Z6 - Z8)/ (4 * cellsize**2)
    G = (Z5 - Z4) / (2 * cellsize)
    H = (Z2 - Z7) / (2 * cellsize)
    plan_curvature = (2 * (D*(H**2) + E*(G**2) - (F*G*H))) / (G**2 + H**2)
    return plan_curvature
def profileCurvature(DEM, cellsize):
    """This process is taken from Change (2014, Introduction to Geographic Information
       systems, Page 284)."""
    Z1 = DEM[0]; Z2 = DEM[1]; Z3 = DEM[2]
    Z4 = DEM[3]; Z0 = DEM[4]; Z5 = DEM[5]
    Z6 = DEM[6]; Z7 = DEM[7]; Z8 = DEM[8]
    D = (((Z4 + Z5)/2) - Z0) / cellsize**2
    E = (((Z2 + Z7)/2) - Z0) / cellsize**2
    F = (Z3 - Z1 + Z6 - Z8)/ (4 * cellsize**2)
    G = (Z5 - Z4) / (2 * cellsize)
    H = (Z2 - Z7) / (2 * cellsize)
    profile_curvature = (-2 * (D*(G**2) + E*(H**2) + (F*G*H))) / (G**2 + H**2)
    return profile_curvature
def circleWindow(radius):
    """Takes a value for radius (r where r is any positive real number) and creates 
       a circular window using that radius."""
    y, x = np.ogrid[-radius: radius + 1, -radius: radius + 1]
    circle = x**2 + y**2 <= radius**2
    return circle
def find_median_value(Window):
    """This function takes the shape function and returns the median value 
        for all valid values (values that fall in the circle) arranged into a 
        1d array. The function also takes the number_of_values function as an input.
        To execute this function type 'find_median_value(shape(r)) where r is any integer.
        #Note: using median like this only gives the correct value for circles with odd 
        radius values."""
    no_values = number_of_values(Window)
    value_range = np.arange(0, no_values + 1)
    central_value = int(np.median(value_range))
    return central_value
def differenceFromMeanElevation(elev):
    """This function only works as an inside function to generic_filter function below. This
       is because generic_filter will take a 2d array and reshape it into a 1d array. Without this 
       step the 'central_value' variable will be outside of the array dimensions. 
       x = input DEM and r = radius of search window """
    centroid = elev[central_value]                        
    mean = np.nanmean(elev)#Count number of values greater than centroid value
    diff = centroid - mean
    return diff
def ghFilter(gully_heads):
    """Reduce the number of candidate initial gully heads. The window size is custimizable through the 
    'footprint' argument. 
    gully_heads == the points found by intersecting drainage lines with profile curvature. """
    max_value = np.max(gully_heads)
    # A small number is being added here just to ensure that the value of the central grid cell is larger than max 
    # if it is the largest value. Just avoiding machine precision issues.
    central_grid_cell = gully_heads[central_value] + 0.01
    if max_value < 0:
        new_value = 0
    elif central_grid_cell >= max_value:
        new_value = 1
    else:
        new_value = 0

    return new_value
def PCTL(x):
    """This function only works as an inside function to generic_filter function below. This
       is because generic_filter will take a 2d array and reshape it into a 1d array. Without this 
       step the 'central_value' variable will be outside of the array dimensions. 
       x = input DEM and r = radius of search window """
    centroid = x[central_value]                        
    y = np.sum(x < centroid)/num_values#Count number of values greater than centroid value
    return y

In [5]:
flow_acc_surf = np.copy(input_DEM).astype('float64');
#################################################################################################
rows = flow_acc_surf.shape[0];
cols = flow_acc_surf.shape[1];
mg = RasterModelGrid((rows,cols), 1);
z1 = mg.add_field('topographic__elevation', flow_acc_surf, at = 'node');

In [6]:
sfb = SinkFillerBarnes(mg, method = 'Steepest', ignore_overfill = True);
sfb.run_one_step();
fa = FlowAccumulator(mg,
                    surface = 'topographic__elevation',
                    flow_director = 'FlowDirectorMFD',
                    diagonals = True);
#(flow_acc, q) = fa.accumulate_flow();
fa.run_one_step();
fd = FlowDirectorMFD(mg, 'topographic__elevation', diagonals = True);
fd.run_one_step();

In [7]:
mg.at_node.keys()

['topographic__elevation',
 'sediment_fill__depth',
 'water__unit_flux_in',
 'flow__link_to_receiver_node',
 'flow__receiver_node',
 'flow__receiver_proportions',
 'topographic__steepest_slope',
 'drainage_area',
 'flow__data_structure_delta',
 'flow__upstream_node_order',
 'surface_water__discharge',
 'flow__sink_flag']

In [8]:
da = np.array(mg.at_node['drainage_area'].round(4))
frn = mg.at_node['flow__receiver_node']
drainage_area = np.flip(da.reshape(mg.shape), 0);
frp = np.array(mg.at_node['flow__receiver_proportions']);
ns = np.flip(mg.status_at_node.reshape(mg.shape), 0)
flow_rec_surf_rows = cols * rows;
flow_rec_surf = frp.reshape(flow_rec_surf_rows,8);

In [9]:
catchment_area = da.reshape(mg.shape) / (10000 * (1/working_resolution**2))

In [10]:
central_value = find_median_value(circleWindow(3))
num_values = number_of_values(circleWindow(3))

In [ ]:
DFME = generic_filter(input_DEM, differenceFromMeanElevation, 
                        footprint= circleWindow(3), mode='nearest');

In [ ]:
dfme_copy = np.copy(DFME)
dfme_threshold = -0.1
dfme_copy[dfme_copy > dfme_threshold] = 0
dfme_copy[dfme_copy < dfme_threshold] = 1

In [ ]:
plt.figure(figsize=(20,10));
plt.imshow(dfme_copy, cmap="gist_earth_r");
plt.colorbar();

In [ ]:
flow_acc_surf = np.copy(input_DEM).astype('float64');
#################################################################################################
rows = flow_acc_surf.shape[0];
cols = flow_acc_surf.shape[1];
mg = RasterModelGrid((rows,cols), 1);
z1 = mg.add_field('topographic__elevation', flow_acc_surf, at = 'node');

In [ ]:
sfb = SinkFillerBarnes(mg, method = 'Steepest', ignore_overfill = True);
sfb.run_one_step();
fa = FlowAccumulator(mg,
                    surface = 'topographic__elevation',
                    flow_director = 'FlowDirectorD8');
#(flow_acc, q) = fa.accumulate_flow();
fa.run_one_step();
fd = FlowDirectorD8(mg, 'topographic__elevation');
fd.run_one_step();

In [ ]:
da = np.array(mg.at_node['drainage_area'].round(4))
frn = mg.at_node['flow__receiver_node']
drainage_area = np.flip(da.reshape(mg.shape), 0);

ns = np.flip(mg.status_at_node.reshape(mg.shape), 0)
flow_rec_surf_rows = cols * rows;


In [ ]:
catchment_area = da.reshape(mg.shape) / (10000 * (1/working_resolution**2))

In [ ]:
plt.figure(figsize=(20,10));
plt.imshow(filtered_gully_drainage, cmap="gist_earth_r");
plt.colorbar();

In [ ]:
gully_internal_drainage = dfme_copy * catchment_area

In [ ]:
shape = 3
central_value = find_median_value(rectangleWindow(shape, shape))
num_values = number_of_values(rectangleWindow(shape, shape))

In [ ]:
pctl_f = filtered_gully_heads_steep = generic_filter(gully_internal_drainage, 
                            PCTL, footprint= rectangleWindow(3,3), mode='constant');

In [ ]:
o = np.copy(pctl_f)
o[o<0.6] = 0
o[o>=0.6] = 1

In [ ]:
concentrated_flow = gully_internal_drainage * o

In [ ]:
min_drainage = np.copy(catchment_area)

In [ ]:
min_drainage[min_drainage<0.1] = 0
min_drainage[min_drainage>0.1] = 1

In [ ]:
filtered_gully_drainage = gully_internal_drainage * min_drainage

In [ ]:
ct = np.copy(concentrated_flow)
ct[ct<0.1] = 0

In [ ]:
steep_drainage_threshold = 0.1

In [ ]:
drainage_copy_steep = np.copy(catchment_area)
drainage_copy_steep[drainage_copy_steep<steep_drainage_threshold]=0
drainage_copy_steep[drainage_copy_steep>steep_drainage_threshold]=1

In [ ]:
flat_drainage_threshold = 0.25

In [ ]:
drainage_copy_flat = np.copy(catchment_area)
drainage_copy_flat[drainage_copy_flat<flat_drainage_threshold]=0
drainage_copy_flat[drainage_copy_flat>flat_drainage_threshold]=1

In [ ]:
central_value = find_median_value(rectangleWindow(9,9))

In [ ]:
slope_resolution = 10
slope_DEM = scipy.ndimage.zoom(input_DEM, input_DEM_resolution / slope_resolution, order = 1)

In [ ]:
slope_coarse = generic_filter(slope_DEM, slopeWindow,
                  footprint= rectangleWindow(3,3),
                  mode='constant', extra_arguments = (slope_resolution,slope_resolution,));

In [ ]:
slope = scipy.ndimage.zoom(slope_coarse, slope_resolution/input_DEM_resolution , order = 1)

In [ ]:
slope[slope > 100] = 100

In [ ]:
flat_areas = np.copy(slope)
flat_areas[flat_areas <= 10] = 1
flat_areas[flat_areas > 10] = 0

In [ ]:
drainage_area_flat = catchment_area * flat_areas
drainage_area_flat[drainage_area_flat < 0.25] = 0
drainage_area_flat[drainage_area_flat >= 0.25] = 1

In [ ]:
steep_areas = np.copy(slope)
steep_areas[steep_areas < 10] = 0
steep_areas[steep_areas >= 10] = 1

In [ ]:
drainage_area_steep = catchment_area * steep_areas
drainage_area_steep[drainage_area_steep < 0.1] = 0
drainage_area_steep[drainage_area_steep >= 0.1] = 1

In [ ]:
initial_gully_heads_flat = drainage_area_flat * dfme_copy

In [ ]:
initial_gully_heads_flat_drainage = initial_gully_heads_flat * catchment_area

In [ ]:
initial_gully_heads_flat_drainage[initial_gully_heads_flat_drainage <= 0] = -10000

In [75]:
central_value = find_median_value(rectangleWindow(3,3))

In [76]:
filtered_gully_heads_flat = generic_filter(filtered_gully_drainage, ghFilter, 
                footprint= rectangleWindow(3,3), 
                mode='constant');


In [ ]:
np.nansum(initial_gully_heads_flat)

In [ ]:
np.nansum(filtered_gully_heads_flat)

In [ ]:
initial_gully_heads_steep = drainage_area_steep * dfme_copy

In [ ]:
initial_gully_heads_steep_drainage = initial_gully_heads_steep * catchment_area

In [ ]:
initial_gully_heads_steep_drainage[initial_gully_heads_steep_drainage <= 0] = -10000

In [ ]:
central_value = find_median_value(rectangleWindow(9,9))

In [ ]:
filtered_gully_heads_steep = generic_filter(initial_gully_heads_steep_drainage, ghFilter, 
                footprint= rectangleWindow(9,9), 
                mode='constant');

In [ ]:
np.nansum(initial_gully_heads_steep)

In [ ]:
np.nansum(filtered_gully_heads_steep)

In [ ]:
all_gully_heads = filtered_gully_heads_steep + filtered_gully_heads_flat

In [ ]:
plt.figure(figsize=(20,10));
plt.imshow(dfme_copy, cmap="gist_earth_r", vmin = -1, vmax = 1);
plt.colorbar();

In [ ]:
h = np.copy(gully_internal_drainage)
h[h<0.25] = 0
h[h>=0.25] = 1

In [ ]:
num_values

In [ ]:
plt.figure(figsize=(20,10));
plt.imshow(ct, cmap="gist_earth_r");
plt.colorbar();

In [ ]:
central_value

In [ ]:
num_values

In [79]:
shape = 3
central_value = find_median_value(rectangleWindow(shape, shape))
num_values = number_of_values(rectangleWindow(shape, shape))

In [80]:
def ghFilter_2(gully_heads):
    """Reduce the number of candidate initial gully heads. The window size is custimizable through the 
    'footprint' argument. """
    surrounding_nodes_list = [];
    for i in range(0, num_values):
        if i != central_value:
            surrounding_nodes_list.append(gully_heads[i])
    
    surrounding_nodes_array = np.array(surrounding_nodes_list)
    non_zero_values = surrounding_nodes_array[surrounding_nodes_array > 0]
    if non_zero_values.size == 0:
        new_value = 0  
    else:
        if gully_heads[central_value] > 0 and np.min(non_zero_values) > gully_heads[central_value]:
            new_value = 1
        else:
            new_value = 0

    return new_value

In [81]:
filtered_gully_head_points = generic_filter(filtered_gully_drainage, 
                            ghFilter_2, footprint= rectangleWindow(shape,shape), mode='constant');

In [84]:
np.sum(filtered_gully_head_points)

253.0

In [85]:
g = np.copy(DFME)
g[g>-0.3] = 0
g[g<=-0.3] = 1

In [87]:
b = g * filtered_gully_head_points

In [88]:
np.sum(b)

24.0

In [26]:
def np_array_to_Geotiff(newfile, original_tiff, np_array, dtype):
    
    cols = np_array.shape[1]
    rows = np_array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = original_tiff.GetGeoTransform() 
    driver = gdal.GetDriverByName('GTiff')
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Float64": 
        GDT_dtype = gdal.GDT_Float64
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")
    
    if np_array.ndim == 2:
        band_num = 1
    else:
        band_num = np_array.shape[2]

    outRaster = driver.Create(newfile, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    
    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
    
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(np_array)
        else:
            outband.WriteArray(np_array[:,:,b])

    # setteing srs from input tif file.
    prj=original_tiff.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    outRaster = None
    
    return outRaster

In [89]:
np_array_to_Geotiff('cgd7.tif', input_geotiff, b, catchment_area.dtype)

C:\Users\sjwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  
